# CAB420, Practical 1 - Question 2 Template
## Linear Regression

Using the dataset from Problem 1, split the data into training, validation and testing as follows:
* Training: All data from the years 2014-2016
* Validation: All data from 2017
* Training: All data from 2018

Develop a regression model to predict one of the cycleway data series in your dataset. In developing this model you should:
* Initially, use all weather data (temperature, rainfall and solar exposure) and all other data series for a particular counter type (i.e. if you’re predicting cyclists inbound for a counter, use all other cyclist inbound counters)
* Use p-values, qqplots, and performance on the validation set to remove terms and improve the model.

When you have finished refining the model, evaluate it on test set, and compare the Root Mean Squared Error (RMSE) for the training, validation and test sets.

In training the model, you will need to ensure that you have no samples (i.e. rows) with missing data. As such, you should remove samples with missing data from the dataset before training and evaluating the model. This may also mean that you have to remove some columns that contain large amounts of missing data.

### Relevant Examples

The first linear regression example, ``CAB420_Regression_Example_1_Linear_Regression.ipynb`` is a useful starting point here.

### Suggested Packages

The following packages are suggested, however there are many ways to approach things in python, if you'd rather use different pacakges that's cool too.

In [1]:
# numpy handles pretty much anything that is a number/vector/matrix/array
import numpy as np
# pandas handles dataframes
import pandas as pd
# matplotlib emulates Matlabs plotting functionality
import matplotlib.pyplot as plt
# seaborn is another good plotting library. In particular, I like it for heatmaps (https://seaborn.pydata.org/generated/seaborn.heatmap.html)
import seaborn as sns;
# stats models is a package that is going to perform the regression analysis
from statsmodels import api as sm
from scipy import stats
from sklearn.metrics import mean_squared_error
# os allows us to manipulate variables on out local machine, such as paths and environment variables
import os
# self explainatory, dates and times
from datetime import datetime, date
# a helper package to help us iterate over objects
import itertools

### Step 1: Load the data
This may be the data you created in Q1, or the pre-baked merged data.

Use pandas and the read_csv function to load the data. It is suggested you inspect the data after loading (print some of it, use the ``head()`` function, possibly plot some series) as a sanity check.

In [12]:
combined = pd.read_csv('combined.csv')
combined['Date'] = pd.to_datetime(combined['Date'])
combined.head()

,Unnamed: 0,Rainfall amount (millimetres),Date,Maximum temperature (Degree C),Daily global solar exposure (MJ/m*m),North Brisbane Bikeway Mann Park Windsor Cyclists Outbound,Jack Pesch Bridge Pedestrians Outbound,Story Bridge East Pedestrian Inbound,Kedron Brook Bikeway Lutwyche Pedestrians Outbound,Kedron Brook Bikeway Mitchelton Pedestrian Outbound,...,Story Bridge East Pedestrian Outbound,North Brisbane Bikeway Mann Park Windsor Pedestrian Outbound,Story Bridge West Cyclists Inbound,Bicenntenial Bikeway,Story Bridge East Cyclists Inbound,Bishop Street Pedestrians Inbound,Story Bridge West Cyclists Outbound,North Brisbane Bikeway Mann Park Windsor Pedestrian Inbound,Kedron Brook Bikeway Mitchelton Pedestrian Inbound,Schulz Canal Bridge Cyclists Inbound
0,0,0.0,2014-01-01,30.6,31.2,NaN,NaN,0.0,NaN,NaN,...,0.0,NaN,0.0,3333.0,0.0,NaN,0.0,NaN,NaN,92.0
1,1,0.0,2014-01-02,31.8,23.4,NaN,NaN,0.0,NaN,NaN,...,0.0,NaN,0.0,4863.0,0.0,NaN,0.0,NaN,NaN,123.0
2,2,1.0,2014-01-03,34.5,29.6,NaN,NaN,0.0,NaN,NaN,...,0.0,NaN,0.0,3905.0,0.0,NaN,0.0,NaN,NaN,77.0
3,3,0.0,2014-01-04,38.7,30.5,NaN,NaN,0.0,NaN,NaN,...,0.0,NaN,0.0,3066.0,0.0,NaN,0.0,NaN,NaN,57.0
4,4,0.0,2014-01-05,33.6,15.7,NaN,NaN,0.0,NaN,NaN,...,0.0,NaN,0.0,4550.0,0.0,NaN,0.0,NaN,NaN,92.0


### Step 2: Filter the data

As you inspect the data, you may see some series have fewer samples than others. Trying to find rows that have all data series may lead to having too little data for analysis. A suggested approach is:
* Loop through the columns in the table. You can use something like ``for column in mydata.columns.values:`` to do this iteration. For each column:
  * Get the number of NaNs in the column. The ``isna()`` function that operates of a pandas series could be useful here.
  * If the column has a number of NaNs above a threshold, flag it for removal
* After the loop, remove the columns. The ``drop()`` function in the pandas dataframe class that takes column names as an input could help here.

Be sure to check what's left in the table after your operations

After this, you should remove any final NaNs. The ``dropna()`` function in the pandas dataframe class could be of use here.

In [13]:
print(np.sum(combined['Date'].isna().to_numpy()))

threshold = 300 
toRemoveColumns = []
for column in combined.columns.values:
        if np.sum(combined[column].isna()) > 300:
            toRemoveColumns.append(column)
            
print(toRemoveColumns) # columns to remove
print(len(toRemoveColumns)) 

combined = combined.drop(columns=toRemoveColumns)

# final removal of any final NaNs
combined = combined.dropna()

combined.head()


0
['North Brisbane Bikeway Mann Park Windsor Cyclists Outbound', 'Jack Pesch Bridge Pedestrians Outbound', 'Kedron Brook Bikeway Lutwyche Pedestrians Outbound', 'Kedron Brook Bikeway Mitchelton Pedestrian Outbound', 'Ekibin Park Pedestrians Outbound', 'Kedron Brook Bikeway Mitchelton', 'Bishop Street Cyclists Inbound', 'Riverwalk Cyclists Inbound', 'Granville Street Bridge Pedestrians Outbound', 'Riverwalk Cyclists Outbound', 'Kedron Brook Bikeway Mitchelton Cyclist Inbound', 'Granville Street Bridge Cyclists Inbound', 'Kedron Brook Bikeway Lutwyche Pedestrians Inbound', 'Ekibin Park Cyclists Inbound', 'Kedron Brook Bikeway Lutwyche Cyclists Inbound', 'Granville Street Bridge Pedestrians Inbound', 'Kedron Brook Bikeway Lutwyche', 'Ekibin Park Cyclists Outbound', 'Ekibin Park Pedestrians Inbound', 'Granville Street Bridge Cyclists Outbound', 'Bishop Street Pedestrians Outbound', 'Riverwalk Pedestrians Inbound', 'Riverwalk Pedestrians Outbound', 'Jack Pesch Bridge Cyclists Inbound', 'Jac

,Unnamed: 0,Rainfall amount (millimetres),Date,Maximum temperature (Degree C),Daily global solar exposure (MJ/m*m),Story Bridge East Pedestrian Inbound,Schulz Canal Bridge Cyclists Outbound,Story Bridge West Pedestrian Outbound,Bicentennial Bikeway Pedestrians Inbound,Story Bridge West Pedestrian Inbound,...,Bicentennial Bikeway Pedestrians Outbound,Story Bridge East Cyclists Outbound,Bicentennial Bikeway Cyclists Outbound,Story Bridge East Pedestrian Outbound,North Brisbane Bikeway Mann Park Windsor Pedestrian Outbound,Story Bridge West Cyclists Inbound,Bicenntenial Bikeway,Story Bridge East Cyclists Inbound,North Brisbane Bikeway Mann Park Windsor Pedestrian Inbound,Schulz Canal Bridge Cyclists Inbound
169,169,0.0,2014-06-19,20.3,8.0,0.0,55.0,0.0,1630.0,0.0,...,1900.0,0.0,333.0,0.0,0.0,0.0,4223.0,0.0,0.0,60.0
170,170,5.8,2014-06-20,22.5,9.1,0.0,49.0,0.0,1170.0,0.0,...,1586.0,0.0,403.0,0.0,0.0,0.0,3619.0,0.0,0.0,45.0
171,171,0.0,2014-06-21,25.6,12.9,0.0,67.0,0.0,1289.0,0.0,...,1847.0,0.0,642.0,0.0,0.0,0.0,4423.0,0.0,0.0,72.0
172,172,5.2,2014-06-22,24.2,13.0,0.0,76.0,0.0,1542.0,0.0,...,2126.0,0.0,635.0,0.0,0.0,0.0,5023.0,0.0,0.0,82.0
173,173,0.2,2014-06-23,24.1,13.6,0.0,69.0,0.0,1862.0,0.0,...,2180.0,0.0,631.0,0.0,0.0,0.0,5329.0,0.0,0.0,74.0


### Step 3: Split into Train, Validation and Test Splits

You can split the data now. Be sure to check dataset size after splitting to make sure that you have datasets of roughly the size you expect.

As part of this you should also pull out your X and Y data, i.e. your predictors and response.

You could also visualise some of this data, and aspects such as:
* Correlation between predictors and the response
* Correlation between pairs of predictors

In [8]:
# Split the data into training, validation and test sets
# The split can be in ratios of 80 10 10 or 70 20 10 
# Where the majority of data should be in the training set

trainData = combined[combined.Date < datetime(year=2017 , month=1 , day=1 )]
validationData = combined[combined.Date >= datetime(year=2017 , month=1 , day=1 ) & combined.Date < datetime(year=2017 , month=12 , day=1 )]
testData = combined[combined.Date >= datetime(year=2018 , month=1 , day=1 )]

# Split the data into X and y
X_train = trainData.drop(columns=['Date', 'Close'])
print(X_train.head())
y_train = trainData['Close']
print(y_train.head())

X_validation = validationData.drop(columns=['Date', 'Close'])
y_validation = validationData['Close']

X_test = testData.drop(columns=['Date', 'Close'])
y_test = testData['Close']




TypeError: unsupported operand type(s) for &: 'datetime.datetime' and 'DatetimeArray'

### Step 4: Create the Model

Using the X and Y arrays you created above, fit a regression model. 

Explore the outputs you get from the model, including:
* The resultant model, including coefficients, p-values, and $R^2$
* A QQ-Plot, to see if assumptions around residuals hold

### Step 5: Refine the Model, and Evaluate the results

Based on model outputs and other data such as correlation, try to improve the model.

Remove terms that look unhelpful. After a term is removed, evaluate the model on the validation and testing sets.